Activate Kafka-HDFS Connectore using Apache Spark

In [ ]:
# !pip install pyspark==4.0.0
# !pip install delta-spark==4.0.0
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
# import os
# os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-17.0.2/"

Buat Session Spark

In [ ]:
spark = SparkSession.builder \
    .appName("KafkatoHDFS") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .getOrCreate()

print("SparkSession build")

PIndahkan data dari Kafka ke HDFS (Data Superstore)

In [ ]:
# Pilih topic yang akan disimpan di storage layer (HDFS)

df = spark.read\
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "twcs") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

df_superstore = df.select(col("value").cast("string").alias("message"), col("timestamp").alias("timestamp_kafka"))
df_superstore.write.mode("overwrite").format("parquet").save("hdfs://localhost:9000/hdfs_twcs")

Cek Hasil di HDFS (Data Superstore)

In [ ]:
sparkcheck = SparkSession.builder.appName("Cek output HDFS").getOrCreate()

df2 = sparkcheck.read.parquet("hdfs://localhost:9000/hdfs_twcs")
df2.show(truncate=False)

Buat Session Spark untuk DeltaSpark

In [ ]:
sparkDelta = SparkSession.builder \
    .appName("DeltaLake") \
    .master("local[*]") \
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0,io.delta:delta-spark_2.13:4.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

print("DeltaSparkSession build")

Pindah format dari dari HDFS (parquet) menjadi format standar data lakehouse (Delta)

In [ ]:
df2 = sparkDelta.read.parquet("hdfs://localhost:9000/hdfs_superstore")
df2.write.format("delta").mode("overwrite").save("hdfs://localhost:9000/delta_superstore")

Cek hasilnya di Delta Lake

In [ ]:
df_delta = sparkDelta.read.format("delta").load("hdfs://localhost:9000/delta_superstore")
df_delta.show()

In [ ]:
df_delta.printSchema()

Data Properties check

In [ ]:
sparkDelta.sql("DESCRIBE DETAIL delta.`hdfs://localhost:9000/delta_superstore`").show()

In [ ]:
sparkDelta.sql("DESCRIBE HISTORY delta.`hdfs://localhost:9000/delta_superstore`").show()